# Exploratory Data Analysis

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df_movies = pd.read_csv("assets/movies.csv")

df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [21]:
df_ratings = pd.read_csv("assets/ratings.csv")

df_ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [22]:
len(df_movies), len(df_ratings)

(58098, 27753444)

In [23]:
# can merge both dataframes by the shared column movieId

df = pd.merge(df_ratings, df_movies, on='movieId')

df

,userId,movieId,rating,timestamp,title,genres
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,6,307,4.0,832059248,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
2,56,307,4.0,1383625728,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,71,307,5.0,1257795414,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
4,84,307,3.0,999055519,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
...,...,...,...,...,...,...
27753439,282403,167894,1.0,1524243885,Stranglehold (1994),Action
27753440,282732,161572,3.5,1504408070,The Great Houdini (1976),Drama
27753441,283000,117857,3.5,1417317969,Hotline (2014),Documentary
27753442,283000,133409,3.5,1431539331,Barnum! (1986),(no genres listed)


In [24]:
len(df["movieId"].unique())

53889

In [25]:
df.info(show_counts = True) # no missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27753444 entries, 0 to 27753443
Data columns (total 6 columns):
 #   Column     Non-Null Count     Dtype  
---  ------     --------------     -----  
 0   userId     27753444 non-null  int64  
 1   movieId    27753444 non-null  int64  
 2   rating     27753444 non-null  float64
 3   timestamp  27753444 non-null  int64  
 4   title      27753444 non-null  object 
 5   genres     27753444 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 1.4+ GB


In [26]:
sorted(df["rating"].unique())

[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]

In [27]:
df["rating"].sort_values()

25152242    0.5
26084402    0.5
16960484    0.5
16960519    0.5
7108785     0.5
           ... 
16833717    5.0
11416495    5.0
24518935    5.0
3320662     5.0
7210728     5.0
Name: rating, Length: 27753444, dtype: float64

In [28]:
df.describe()

,userId,movieId,rating,timestamp
count,2.775344e+07,2.775344e+07,2.775344e+07,2.775344e+07
mean,1.419420e+05,1.848800e+04,3.530445e+00,1.193122e+09
std,8.170740e+04,3.510263e+04,1.066353e+00,2.160482e+08
min,1.000000e+00,1.000000e+00,5.000000e-01,7.896520e+08
25%,7.117600e+04,1.097000e+03,3.000000e+00,9.986053e+08
50%,1.420220e+05,2.716000e+03,3.500000e+00,1.174256e+09
75%,2.124590e+05,7.150000e+03,4.000000e+00,1.422744e+09
max,2.832280e+05,1.938860e+05,5.000000e+00,1.537945e+09


TODO: this will be moved to the actual recommendation system later

# Find user's most watched genre

In [29]:
def extract_genres(df):
    genre_counts = {}

    # iterate over the selected rows
    for genres in df['genres']:
        # split the genres string into a list of individual genre strings
        genre_list = genres.split('|')
        # iterate over the genre strings
        for genre in genre_list:
            # check if the genre is already in the dictionary
            if genre in genre_counts:
                # if so, increment its count
                genre_counts[genre] += 1
            else:
                # if not, add it to the dictionary with a count of 1
                genre_counts[genre] = 1
    return genre_counts

In [30]:
def generate(df, userId, threshold, results):
    # pick out movies user has watched with a rating over threshold
    selected_df = df[(df['userId'] == userId) & (df['rating'] > threshold)]

    genre_counts = extract_genres(selected_df)

    sorted_dict = dict(sorted(genre_counts.items(), key=lambda x: x[1], reverse = True))
    
    top = dict(list(sorted_dict.items())[:results])
    return top

In [31]:
user = 4 # userId
lowest_rating = 3 # only take into account movies with a rating above this value
top = 4 # amount of top genres to return

generate(df, user, lowest_rating, top)

{'Action': 248, 'Thriller': 201, 'Drama': 179, 'Adventure': 145}

In [32]:
df[df["genres"].str.contains("Action") | df["genres"].str.contains("Thriller")]

,userId,movieId,rating,timestamp,title,genres
7958,1,481,3.5,1256677456,Kalifornia (1993),Drama|Thriller
7959,4,481,4.0,1204592835,Kalifornia (1993),Drama|Thriller
7960,19,481,3.0,836845062,Kalifornia (1993),Drama|Thriller
7961,38,481,1.0,982855311,Kalifornia (1993),Drama|Thriller
7962,114,481,3.0,973440263,Kalifornia (1993),Drama|Thriller
...,...,...,...,...,...,...
27753419,282119,101483,3.5,1364434056,"Likeable Mister R, The (Simpaticul domn R) (1969)",Action|Crime|Thriller
27753424,282119,173213,3.0,1495495609,The Goodbye Kiss (2006),Crime|Drama|Thriller
27753428,282119,177869,3.5,1515539588,Girls und Panzer Movie (2015),Action|Animation|Comedy
27753431,282374,168674,1.5,1486168497,Amsterdam Heavy (2011),Action


---

TODO: these might go in a separate data cleaning file later:

# Drop users with less than N ratings

In [33]:
def drop_users(df, min_ratings, max_ratings):
    counts = df['userId'].value_counts()

    # filter out users with less than min_ratings (less relevant due to not having many movie recommendations)
    df_filtered = df[df["userId"].isin(counts[counts >= min_ratings].index)]

    # filter out users with more than max_ratings (outlier data)
    # (also less relevant due to having too many movie recommendations, and high likelihood of spam bots)
    df_filtered = df_filtered[df_filtered["userId"].isin(counts[counts <= max_ratings].index)]

    return df_filtered

In [34]:
df_filtered = drop_users(df, 4, 1000)

original = len(df['userId'].unique())
new = len(df_filtered['userId'].unique())

print(f"Unique users of dataset reduced from {original:,} to {new:,} (a reduction of {original - new:,} users or {(original - new) / original * 100:.2f}%)\n")

original = len(df)
new = len(df_filtered)

print(f"Length of dataset reduced from {original:,} to {new:,} (a reduction of {original - new:,} rows or {(original - new) / original * 100:.2f}%)")

Unique users of dataset reduced from 283,228 to 267,349 (a reduction of 15,879 users or 5.61%)

Length of dataset reduced from 27,753,444 to 23,441,845 (a reduction of 4,311,599 rows or 15.54%)


In [35]:
df_filtered["userId"].value_counts() # only users with between 1000 and 4 ratings left in dataset

77054     1000
88187     1000
151764    1000
268277    1000
10355     1000
          ... 
257011       4
261569       4
270783       4
276520       4
251924       4
Name: userId, Length: 267349, dtype: int64

# Dropping users with a low average rating

In [36]:
def drop_low_average(df, min_average):

    # Get the mean rating for each user
    user_mean_ratings = df.groupby('userId')['rating'].mean()

    # Get the userIds with mean rating above N
    user_ids_to_keep = user_mean_ratings[user_mean_ratings >= min_average].index

    # Filter the DataFrame to keep only the desired userIds
    df_filtered = df[df['userId'].isin(user_ids_to_keep)]

    return df_filtered

In [37]:
df_filtered = drop_low_average(df, 2)

original = len(df['userId'].unique())
new = len(df_filtered['userId'].unique())

print(f"Unique users of dataset reduced from {original:,} to {new:,} (a reduction of {original - new:,} users or {(original - new) / original * 100:.2f}%)\n")

original = len(df)
new = len(df_filtered)

print(f"Length of dataset reduced from {original:,} to {new:,} (a reduction of {original - new:,} rows or {(original - new) / original * 100:.2f}%)")

Unique users of dataset reduced from 283,228 to 280,069 (a reduction of 3,159 users or 1.12%)

Length of dataset reduced from 27,753,444 to 27,588,915 (a reduction of 164,529 rows or 0.59%)


# Removing rows with low scores

In [38]:
def drop_ratings(df, min_rating):

    # removing scores of less than min_rating (not relevant to recommend to anyone)
    df_filtered = df.loc[df['rating'] >= min_rating]

    return df_filtered

In [39]:
df_filtered = drop_ratings(df, 2.5)

original = len(df['userId'].unique())
new = len(df_filtered['userId'].unique())

print(f"Unique users of dataset reduced from {original:,} to {new:,} (a reduction of {original - new:,} users or {(original - new) / original * 100:.2f}%)\n")

original = len(df)
new = len(df_filtered)

print(f"Length of dataset reduced from {original:,} to {new:,} (a reduction of {original - new:,} rows or {(original - new) / original * 100:.2f}%)")

Unique users of dataset reduced from 283,228 to 282,009 (a reduction of 1,219 users or 0.43%)

Length of dataset reduced from 27,753,444 to 24,132,842 (a reduction of 3,620,602 rows or 13.05%)


# Dropping all at once

In [40]:
df_filtered = drop_users(df, 4, 1000)
df_filtered = drop_low_average(df_filtered, 2)
df_filtered = drop_ratings(df_filtered, 2.5)

df_filtered

,userId,movieId,rating,timestamp,title,genres
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,6,307,4.0,832059248,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
2,56,307,4.0,1383625728,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,71,307,5.0,1257795414,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
4,84,307,3.0,999055519,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
...,...,...,...,...,...,...
27753413,281447,193805,2.5,1537645510,95 (2017),Comedy|Drama
27753415,281983,159618,4.0,1464570010,Snake Dancer (1976),Drama
27753416,281983,159620,3.5,1464574286,Angel of Destruction (1994),Action|Adventure|Thriller
27753417,281983,159660,3.0,1464710418,One Girl's Confession (1953),Drama


In [41]:
original = len(df)
new = len(df_filtered)

print(f"Length of dataset reduced from {original:,} to {new:,} (a reduction of {original - new:,} rows or {(original - new) / original * 100:.2f}%)")

Length of dataset reduced from 27,753,444 to 20,549,358 (a reduction of 7,204,086 rows or 25.96%)


# Way of manually looking at movies

In [42]:
title_substring = "the fountain"

# filter out titles containing the string
titles = df[df['title'].str.contains(title_substring, case = False)]['title'].unique().tolist()

# print out information about each title matching the substring
for title in titles:
    print(f"{title}\nRating: {df[df['title'] == title]['rating'].mean():.1f}\nOccurrences = {len(df[df['title'] == title]):,}\n")

The Fountain (1989)
Rating: 3.5
Occurrences = 4

Three Coins in the Fountain (1954)
Rating: 2.8
Occurrences = 33

The Fountain of Youth (1958)
Rating: 3.7
Occurrences = 3

